In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
folder_path = f'C:/Users/user/PycharmProjects/PersianChildhood'
driver_path = 'C:/Users/User/Downloads/chromedriver_win32 (3)/chromedriver.exe'
page_url = f'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview'
base_url = 'https://digitale-sammlungen.ulb.uni-bonn.de/topic/view/3080607'

In [ ]:
#get all journals on a base_url page

#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(base_url) 
#get list of all journal links
meta_data_dfs = []
journals = driver.find_elements_by_class_name('title')
journal_count = len(journals)
for count in range(journal_count):    
    #go to journal homepage, get title and create directory   
    journal = driver.find_elements_by_class_name('title')[count]
    journal_name = journal.text
    journal_name = journal_name.replace(':','-')
    journal_name = journal_name.replace('/','-')
    journal.click()
    time.sleep(3)
#     journal_name  = driver.find_elements_by_class_name('valueDiv')[0].text
    #go to first page of journal
    driver.find_element_by_class_name('imgLink').click()   
    time.sleep(3)
    #get list of all pages
    page_numbers = []
    page_names = []
    page_links = []
    mag_names = []
    volume_names = []
    ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
    for i in ls:
        page_numbers.append(i.get_attribute(name='value'))
        page_names.append(i.text.split()[-1])
    #for each page, open it, get the details of the page, and then download the full image
    temp_mag_name = ''
    temp_count = 1
    for ind in range(len(ls)):
        page_link = f'{page_url}/{page_numbers[ind]}'
        driver.get(page_link)
        page_links.append(page_link)        
        time.sleep(3)
        volume_name = driver.find_element_by_class_name('journal_volume').text
        volume_name = volume_name.replace('/','-') #remove unformatted char        
        tryos.makedirs(f"{folder_path}/{journal_name}/{volume_name}/{new_mag_name}", exist_ok=True)
        volume_names.append(volume_name)

        try:
            driver.find_element_by_id('open-pageviewStructure-tree').click()
        except: #try again after refreshing
            driver.refresh()
            time.sleep(3)
            driver.find_element_by_id('open-pageviewStructure-tree').click()            
        for page in driver.find_elements_by_class_name('pagelink'):
            if page_numbers[ind] in page.get_attribute('href'):
                if len(page.text.split('\n'))>1:
                    new_mag_name = page.text.split('\n')[1]
                else:
                    new_mag_name = page.text                    
        new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
        #mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[2].text
        new_mag_name = new_mag_name.replace('/','-') #remove unformatted char        
        mag_names.append(new_mag_name)
        #for a given page, go to its full image
        os.makedirs(f"{folder_path}/{journal_name}/{volume_name}/{new_mag_name}", exist_ok=True)
        if new_mag_name==temp_mag_name:
            temp_count=temp_count+1
        else:
            temp_mag_name = new_mag_name
            temp_count=1
        urllib.request.urlretrieve(new_url,f"{folder_path}/{journal_name}/{volume_name}/{new_mag_name}/p_{temp_count} ({page_names[ind]}).jpg")
        #create meta_data
    meta_data_df = pd.DataFrame({'journal':journal_name,'volume':volume_names,'issue':mag_names,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})
    # and now go back to base url
    driver.get(base_url) 
meta_data_dfs.append(meta_data_df)
driver.close()

In [125]:
#old version
#get all journals on a base_url page

#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(base_url) 
#get list of all journal links
meta_data_dfs = []
journals = driver.find_elements_by_class_name('title')
journal_count = len(journals)
for count in range(journal_count):    
    #go to journal homepage, get title and create directory   
    journal = driver.find_elements_by_class_name('title')[count]
    journal_name = journal.text
    journal_name = journal_name.replace(':','-')
    journal_name = journal_name.replace('/','-')
    try:
        os.makedirs(f"{folder_path}/{journal_name}", exist_ok=False)
    except:
        continue
    journal.click()
    time.sleep(3)
#     journal_name  = driver.find_elements_by_class_name('valueDiv')[0].text
    #go to first page of journal
    driver.find_element_by_class_name('imgLink').click()   
    time.sleep(3)
    #get list of all pages
    page_numbers = []
    page_names = []
    page_links = []
    mag_names = []
    volume_names = []
    ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
    for i in ls:
        page_numbers.append(i.get_attribute(name='value'))
        page_names.append(i.text.split()[-1])
    #for each page, open it, get the details of the page, and then download the full image
    temp_mag_name = ''
    temp_count = 1
    for ind in range(len(ls)):
        page_link = f'{page_url}/{page_numbers[ind]}'
        driver.get(page_link)
        page_links.append(page_link)
        time.sleep(3)
        try:
            driver.find_element_by_id('open-pageviewStructure-tree').click()
        except: #try again after refreshing
            driver.refresh()
            time.sleep(3)
            driver.find_element_by_id('open-pageviewStructure-tree').click()            
        for page in driver.find_elements_by_class_name('pagelink'):
            if page_numbers[ind] in page.get_attribute('href'):
                if len(page.text.split('\n'))>1:
                    new_mag_name = page.text.split('\n')[1]
                else:
                    new_mag_name = page.text                    
        new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
        #mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[2].text
        new_mag_name = new_mag_name.replace('/','-') #remove unformatted char        
        mag_names.append(new_mag_name)
        volume_name = driver.find_element_by_class_name('journal_volume').text
        volume_name = volume_name.replace('/','-') #remove unformatted char        
        volume_names.append(volume_name)
        #for a given page, go to its full image
        os.makedirs(f"{folder_path}/{journal_name}/{volume_name}/{new_mag_name}", exist_ok=True)
        if new_mag_name==temp_mag_name:
            temp_count=temp_count+1
        else:
            temp_mag_name = new_mag_name
            temp_count=1
        urllib.request.urlretrieve(new_url,f"{folder_path}/{journal_name}/{volume_name}/{new_mag_name}/p_{temp_count} ({page_names[ind]}).jpg")
        #create meta_data
    meta_data_df = pd.DataFrame({'journal':journal_name,'volume':volume_names,'issue':mag_names,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})
    # and now go back to base url
    driver.get(base_url) 
meta_data_dfs.append(meta_data_df)
driver.close()

KeyboardInterrupt: 

In [139]:
for page in driver.find_elements_by_class_name('pagelink'):
    if page_numbers[ind] in page.get_attribute('href'):
        if len(page.text.split('\n'))>1:
            new_mag_name = page.text.split('\n')[1]
        else:
            new_mag_name = page.text
new_mag_name

'Inhaltsverzeichnis und Register'

In [143]:
page_numbers[ind]

'7083049'

In [144]:
page.get_attribute('href')

'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/7032526'

In [137]:
page.text

'2. Jahrgang (Šaʿbān 1339 - Ǧumādā l-ūlā 1341hqš // Esfand 1321h)'

In [151]:
driver.find_elements_by_class_name("npNext")[0].text

''

In [124]:
driver.close()


In [14]:
driver.find_element_by_id('open-pageviewStructure-tree').click()


In [50]:
driver.find_elements_by_class_name('paginationcaption')[-2].is_selected()

False

In [74]:
driver.find_elements_by_class_name('pagelink')[-8].text

'41 \nNummer 11 (30. Ḏū l-ḥiǧǧa 1292 // 13.[sic!] Kânûn-ı Sânî 1291 // 25. [sic !] Januar 1876)'

In [53]:
driver.find_element_by_tag_name('a')

<selenium.webdriver.remote.webelement.WebElement (session="c3c1a5b223b931c2f3994ff36d4ca0b4", element="EB3903B4BC87FC95D16755C65A7D38FC_element_1039")>

In [123]:
new_mag_name

'Inhaltsverzeichnis und Register'

In [79]:
driver.find_element_by_id('open-pageviewStructure-tree').click()
for page_link in driver.find_elements_by_class_name('pagelink'):
    if page_numbers[ind] in page_link.get_attribute('href'):
        new_mag_name = page_link.text.replace('\n','')
new_mag_name

'41 Nummer 11 (30. Ḏū l-ḥiǧǧa 1292 // 13.[sic!] Kânûn-ı Sânî 1291 // 25. [sic !] Januar 1876)'

In [77]:
print(new_mag_name)

41 
Nummer 11 (30. Ḏū l-ḥiǧǧa 1292 // 13.[sic!] Kânûn-ı Sânî 1291 // 25. [sic !] Januar 1876)


In [ ]:
meta_data_master = pd.concat(meta_data_dfs)
meta_data_master.to_excel('metadata master.xlsx')

In [14]:
#get single journal

# 4287066 : Sur-Israfil
# 3275358 : Ahtar
# 7265310 : Maglis
# 7050077 : Habl al-Matin
first_page='7050077'
full_url = f'{url}/{first_page}'
name = 'Habl al-Matin'

In [24]:
chrome = webdriver.ChromeOptions()
#chrome.add_argument("--headless=new")
#chrome.headless = True
chrome.add_experimental_option("prefs", {'download.default_directory': folder_path})    
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(full_url)
#get list of all pages
page_numbers = []
page_names = []
ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
for i in ls:
    page_numbers.append(i.get_attribute(name='value'))
    page_names.append(i.text.split()[-1])
#for each page, open it, get the name of the magazine, and then download the full image
for ind in range(len(ls)):
    driver.get(f'{url}/{page_numbers[ind]}')
    time.sleep(3)
    new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
    mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[2].text
    mag_name = mag_name.replace('/','-') #remove unformatted char
    #for a given page, go to its full image
    urllib.request.urlretrieve(new_url,f"{folder_path}/{name}/{mag_name}_{page_names[ind]}.jpg")

In [28]:

links = driver.find_elements_by_class_name('title')
links[0].click()

In [40]:
journal_name  = driver.find_elements_by_class_name('valueDiv')[0].text
os.mkdir(journal_name)

In [20]:
driver.find_element_by_class_name('imgLink').click()

In [19]:
driver.find_element_by_class_name('imgLink').get_attribute('href')

'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3275358'

In [55]:
meta_data_df.link[0]

'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/7059681'

In [58]:
meta_data_df = pd.DataFrame({'journal':journal_name,'volume':volume_names,'issue':mag_names,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})
meta_data_df

,journal,volume,issue,page,link,extraction date
0,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),[1],https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
1,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),2,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
2,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),3,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
3,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),4,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
4,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),5,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
5,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),6,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
6,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),7,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
7,Šarāfat,1. Jahrgang (1314-1321hq),1. Ausgabe (Ṣafar 1314),8,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
8,Šarāfat,1. Jahrgang (1314-1321hq),2. Ausgabe (Rabīʿ al-awwal 1314),[9],https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22
9,Šarāfat,1. Jahrgang (1314-1321hq),2. Ausgabe (Rabīʿ al-awwal 1314),10,https://digitale-sammlungen.ulb.uni-bonn.de/ul...,2023-06-22


In [40]:
page_names

['[1]',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '[9]',
 '10',
 '11',
 '12',
 '[13]',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '[21]',
 '22',
 '23',
 '24',
 '[25]',
 '26',
 '27',
 '28',
 '[29]',
 '30',
 '31',
 '32',
 '[33]',
 '34',
 '35',
 '36',
 '[37]',
 '30',
 '31',
 '32',
 '[41]',
 '34',
 '35',
 '36',
 '[45]',
 '38',
 '39',
 '40',
 '[49]',
 '50',
 '51',
 '52',
 '[53]',
 '54',
 '55',
 '56',
 '[57]',
 '58',
 '59',
 '60',
 '[61]',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '[69]',
 '70',
 '71',
 '72',
 '[73]',
 '74',
 '75',
 '76',
 '[77]',
 '78',
 '79',
 '80',
 '[81]',
 '82',
 '83',
 '84',
 '[85]',
 '86',
 '87',
 '88',
 '[89]',
 '90',
 '91',
 '92',
 '[93]',
 '94',
 '95',
 '96',
 '[97]',
 '98',
 '99',
 '100',
 '[101]',
 '102',
 '103',
 '104',
 '[105]',
 '106',
 '107',
 '108',
 '[109]',
 '110',
 '111',
 '112',
 '[113]',
 '114',
 '115',
 '116',
 '[117]',
 '118',
 '119',
 '120',
 '[121]',
 '122',
 '123',
 '124',
 '[125]',
 '126',
 '127',
 '128',
 '[129]',
 '130',
 '131',


In [31]:
page_names 

['[1]',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '[9]',
 '10',
 '11',
 '12',
 '[13]',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '[21]',
 '22',
 '23',
 '24',
 '[25]',
 '26',
 '27',
 '28',
 '[29]',
 '30',
 '31',
 '32',
 '[33]',
 '34',
 '35',
 '36',
 '[37]',
 '30',
 '31',
 '32',
 '[41]',
 '34',
 '35',
 '36',
 '[45]',
 '38',
 '39',
 '40',
 '[49]',
 '50',
 '51',
 '52',
 '[53]',
 '54',
 '55',
 '56',
 '[57]',
 '58',
 '59',
 '60',
 '[61]',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '[69]',
 '70',
 '71',
 '72',
 '[73]',
 '74',
 '75',
 '76',
 '[77]',
 '78',
 '79',
 '80',
 '[81]',
 '82',
 '83',
 '84',
 '[85]',
 '86',
 '87',
 '88',
 '[89]',
 '90',
 '91',
 '92',
 '[93]',
 '94',
 '95',
 '96',
 '[97]',
 '98',
 '99',
 '100',
 '[101]',
 '102',
 '103',
 '104',
 '[105]',
 '106',
 '107',
 '108',
 '[109]',
 '110',
 '111',
 '112',
 '[113]',
 '114',
 '115',
 '116',
 '[117]',
 '118',
 '119',
 '120',
 '[121]',
 '122',
 '123',
 '124',
 '[125]',
 '126',
 '127',
 '128',
 '[129]',
 '130',
 '131',


In [80]:
driver.close()

In [ ]:
#get list of all pages
page_numbers = []
page_names = []
ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
for i in ls:
    page_numbers.append(i.get_attribute(name='value'))
    page_names.append(i.text.split()[-1])
#for each page, open it, get the name of the magazine, and then download the full image
for ind in range(len(ls)):
    driver.get(f'{url}/{page_numbers[ind]}')
    time.sleep(3)
    new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
    mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[2].text
    mag_name = mag_name.replace('/','-') #remove unformatted char
    #for a given page, go to its full image
    urllib.request.urlretrieve(new_url,f"{folder_path}/{name}/{mag_name}_{page_names[ind]}.jpg")